In [10]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# скачиваем данные и разделяем на набор для обучения и тестовый
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)

num_classes = 10
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# преобразование векторных классов в бинарные матрицы
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('Размерность x_train:', x_train.shape)
print(x_train.shape[0], 'Размер train')
print(x_test.shape[0], 'Размер test')

# используем сверточную нейронную сеть
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

batch_size = 128
epochs = 10

hist = model.fit(x_train, y_train, batch_size = batch_size, epochs=epochs, verbose=1, validation_data=(x_test, y_test))
print("Модель успешно обучена")

model.save('mnist.h5')
print("Модель сохранена как mnist.h5")

score = model.evaluate(x_test, y_test, verbose=0)
print('Потери на тесте:', score[0])
print('Точность на тесте:', score[1])

(60000, 28, 28) (60000,)
Размерность x_train: (60000, 28, 28, 1)
60000 Размер train
10000 Размер test
Epoch 1/10
469/469 [==============================] - 151s 321ms/step - loss: 2.2829 - accuracy: 0.1310 - val_loss: 2.2310 - val_accuracy: 0.2739
Epoch 2/10
469/469 [==============================] - 152s 324ms/step - loss: 2.2229 - accuracy: 0.2354 - val_loss: 2.1496 - val_accuracy: 0.4063
Epoch 3/10
469/469 [==============================] - 153s 326ms/step - loss: 2.1414 - accuracy: 0.3456 - val_loss: 2.0364 - val_accuracy: 0.5559
Epoch 4/10
469/469 [==============================] - 153s 327ms/step - loss: 2.0316 - accuracy: 0.4465 - val_loss: 1.8790 - val_accuracy: 0.6362
Epoch 5/10
469/469 [==============================] - 153s 327ms/step - loss: 1.8743 - accuracy: 0.5356 - val_loss: 1.6705 - val_accuracy: 0.6967
Epoch 6/10
469/469 [==============================] - 153s 327ms/step - loss: 1.6824 - accuracy: 0.5949 - val_loss: 1.4268 - val_accuracy: 0.7477
Epoch 7/10
469/469 [==

In [11]:
%shell
!pip install --no-deps tensorflowjs
!rm -rf mnist-draw mode
!tensorflowjs_converter --input_format keras mnist.h5 model/
!git clone https://github.com/marcacohen/mnist-draw.git
!rm -rf mnist-draw/model
!mv model mnist-draw
!cd mnist-draw/
!pip install -q -r requirements.txt

import http.server
import socketserver
import portpicker
import threading
import os
from google.colab import output
from IPython.display import Javascript

started = threading.Event()
port = portpicker.pick_unused_port()

# Start a background thread serving up the files.
def server_entry():
  os.chdir('/content/mnist-draw')
  with socketserver.TCPServer(("", port), http.server.SimpleHTTPRequestHandler) as httpd:
    started.set()
    httpd.serve_forever()

thread = threading.Thread(target=server_entry)
thread.start()
started.wait();

output.eval_js(f"window.mnistPort = {port}")

Javascript('''
  (async () => {
    if (!google.colab.kernel.accessAllowed) {
        return;
    }
    const url = await google.colab.kernel.proxyPort(window.mnistPort);
    const iframe = document.createElement('iframe');
    iframe.src = url;
    iframe.height = '500px';
    iframe.width = '100%';
    iframe.style.border = '0';
    document.body.appendChild(iframe);
  })()
''')

     |████████████████████████████████| 71kB 3.7MB/s 
2020-12-30 08:46:04.162501: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Cloning into 'mnist-draw'...
remote: Enumerating objects: 297, done.
remote: Total 297 (delta 0), reused 0 (delta 0), pack-reused 297
Receiving objects: 100% (297/297), 15.14 MiB | 29.03 MiB/s, done.
Resolving deltas: 100% (134/134), done.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<IPython.core.display.Javascript object>